# TEST / SQLite

In [1]:
import numpy as np
import pandas as pd
import sqlite3
import timeit
import datetime as dt


# Get data

In [2]:
URLname = ""
Filename = ""


In [3]:
#import requests
#r = requests.get(url=URLname)
#data = r.json()


In [4]:
df = pd.read_csv(Filename)
print("df.shape = ", df.shape)
df.head()


/Users/mycomputer/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


df.shape =  (8726014, 19)


,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,1103341116,2015-12-21T00:00:00,1251.0,NaN,NaN,CA,200304.0,NaN,NaN,PA,GY,13147 WELBY WAY,01521,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0
1,1103700150,2015-12-21T00:00:00,1435.0,NaN,NaN,CA,201512.0,NaN,NaN,VN,WH,525 S MAIN ST,1C51,1.0,4000A1,NO EVIDENCE OF REG,50.0,99999.0,99999.0
2,1104803000,2015-12-21T00:00:00,2055.0,NaN,NaN,CA,201503.0,NaN,NaN,PA,BK,200 WORLD WAY,2R2,2.0,8939,WHITE CURB,58.0,6439997.9,1802686.4
3,1104820732,2015-12-26T00:00:00,1515.0,NaN,NaN,CA,NaN,NaN,NaN,PA,WH,100 WORLD WAY,2F11,2.0,000,17104h,NaN,6440041.1,1802686.2
4,1105461453,2015-09-15T00:00:00,115.0,NaN,NaN,CA,200316.0,NaN,CHEV,PA,BK,GEORGIA ST/OLYMPIC,1FB70,1.0,8069A,NO STOPPING/STANDING,93.0,99999.0,99999.0


### Replace spaces with "_" in pandas dataframe

In [5]:
# Original names
list_cols = list(df.columns)
#list_cols


In [6]:
# Names with spaces replaced by underscores
list_cols_new = []
for c in df.columns:
    c1 = c.replace(" ","_")
    list_cols_new.append(c1)

#list_cols_new


In [7]:
# Create before/after dictionary.  (Simpler to include ones that don't change)
d_change = {}
for c1,c2 in zip(list_cols,list_cols_new):
    d_change[c1] = c2
#d_change    


In [8]:
# Now make the change
df.rename( columns=d_change, inplace=True)


In [9]:
# The new names are...
df.columns

Index(['Ticket_number', 'Issue_Date', 'Issue_time', 'Meter_Id', 'Marked_Time',
       'RP_State_Plate', 'Plate_Expiry_Date', 'VIN', 'Make', 'Body_Style',
       'Color', 'Location', 'Route', 'Agency', 'Violation_code',
       'Violation_Description', 'Fine_amount', 'Latitude', 'Longitude'],
      dtype='object')

# SQLite

In [10]:
con = sqlite3.connect("test.db")
cur = con.cursor()

In [11]:
# If I've already created this table (while developing)...
cur.execute("DROP TABLE IF EXISTS mytest")


## Create table from pandas dataframe

In [12]:
# Create table from pandas dataframe
df.to_sql("mytest",con,index=False)


In [13]:
# list tables in database
#for row in cur.execute("SELECT name FROM sqlite_master"):
#    print(row)


In [14]:
# List tables
#for row in cur.execute("SELECT name FROM PRAGMA_TABLE_INFO('mytest')"):
#    print(row)
    

In [15]:
# Just checking it works...
#for row in cur.execute("SELECT Make,Agency,Violation_code FROM mytest LIMIT 20"):
#    print(row)


# Problem (a) ----------------------------------------------------

### SQLite

In [16]:
time1 = timeit.default_timer()

for row in cur.execute("SELECT Make, COUNT(*) AS cnt FROM mytest GROUP BY Make HAVING Make IS NOT NULL ORDER BY cnt DESC LIMIT 25"):
    print(row)
    
time2 = timeit.default_timer()
print("time = ", (time2 - time1) )
    

('TOYT', 721411)
('HOND', 491961)
('FORD', 382695)
('NISS', 311324)
('CHEV', 297076)
('BMW', 199221)
('MERZ', 177307)
('VOLK', 149501)
('HYUN', 133864)
('DODG', 127764)
('LEXS', 124508)
('KIA', 101746)
('JEEP', 100909)
('AUDI', 84229)
('MAZD', 79853)
('OTHR', 72411)
('GMC', 62391)
('CHRY', 57317)
('INFI', 56809)
('ACUR', 52703)
('SUBA', 46898)
('VOLV', 42330)
('TOYO', 40064)
('MITS', 37842)
('CADI', 34080)
time =  2.962376507028239


### Pandas

In [17]:
# Pandas

time1 = timeit.default_timer()

# Include Make==null
#df_top25 = pd.DataFrame(df['Make'].value_counts(dropna=False)).head(25)
#df_top25

# Exclude Make==null
df_top25 = pd.DataFrame(df['Make'].value_counts()).head(25)
print(df_top25)

time2 = timeit.default_timer()
print("time = ", (time2 - time1) )


        Make
TOYT  721411
HOND  491961
FORD  382695
NISS  311324
CHEV  297076
BMW   199221
MERZ  177307
VOLK  149501
HYUN  133864
DODG  127764
LEXS  124508
KIA   101746
JEEP  100909
AUDI   84229
MAZD   79853
OTHR   72411
GMC    62391
CHRY   57317
INFI   56809
ACUR   52703
SUBA   46898
VOLV   42330
TOYO   40064
MITS   37842
CADI   34080
time =  0.7391566729929764


### Results:  SQLite 2.96s  /  Pandas 0.73s

# Problem (b) ----------------------------------------------------

### SQLite

In [18]:
time1 = timeit.default_timer()

# This is the count for each Make/Color combo
#for row in cur.execute("SELECT Make, Color, COUNT(*) AS cnt FROM mytest GROUP BY Make, Color HAVING Make IS NOT NULL ORDER BY Make,Color,cnt DESC"):
#    print(row)

# This is the Make/Color/count combo having the maximum count for that Make (ie, the most popular Color for that Make)
for row in cur.execute("SELECT Make,Color,MAX(cnt) FROM (SELECT Make, Color, COUNT(*) AS cnt FROM mytest GROUP BY Make, Color HAVING Make IS NOT NULL) GROUP BY Make"):
    print(row)

time2 = timeit.default_timer()
print("\ntime = ", (time2 - time1) )



('AAUD', 'BK', 1)
('AAVH', 'WH', 2)
('ABAR', 'BK', 3)
('ABC', 'WH', 3)
('ABRI', 'GY', 1)
('ACC', 'GY', 1)
('ACCR', 'GR', 1)
('ACCU', 'BK', 3)
('ACDI', 'BK', 1)
('ACG', 'WH', 2)
('ACOR', 'GY', 1)
('ACRA', 'BK', 4)
('ACRU', 'BK', 1)
('ACU', 'BK', 10)
('ACUA', 'SI', 1)
('ACUR', 'BK', 13041)
('ADIA', 'WH', 1)
('ADVE', 'BL', 1)
('AERO', 'WH', 1)
('AFLA', 'RE', 1)
('AIRB', 'WH', 1)
('AIRE', 'TA', 4)
('AIRS', 'WH', 2)
('AIRX', 'BE', 1)
('AJAX', 'BL', 8)
('ALAX', 'GY', 1)
('ALEG', 'WH', 7)
('ALER', 'WH', 2)
('ALFA', 'BK', 202)
('ALGR', 'WH', 2)
('ALJD', 'WH', 1)
('ALJO', 'WH', 2)
('ALLD', 'BK', 1)
('ALLE', 'TA', 4)
('ALLG', 'BE', 1)
('ALLY', 'BK', 2)
('ALPE', 'WH', 1)
('ALPH', 'GY', 2)
('ALTI', 'GR', 1)
('ALTM', 'SI', 1)
('AM', 'GY', 1)
('AMC', 'WH', 4)
('AMCA', 'GR', 1)
('AMCE', 'VA', 1)
('AMCL', 'WH', 6)
('AMER', 'BK', 6)
('ANDE', 'BE', 1)
('APAC', 'WH', 1)
('APCH', 'BK', 2)
('APOL', 'WH', 3)
('APPL', 'WH', 1)
('APPO', 'WH', 1)
('APR', 'RE', 1)
('APRI', 'YE', 1)
('APRL', 'BK', 1)
('AREB', 'T

### Pandas

In [19]:
time1 = timeit.default_timer()

# Only look at Make != null
filt = ~df['Make'].isnull()
df2 = df.loc[ filt, ['Make','Color'] ]

# Get the number in each Make/Color group
x = pd.DataFrame( df2.groupby(['Make','Color']).size() )
x2 = x.reset_index()

# Get rid of that 0 column
x2.columns
x2.rename( columns={0:'cnt'}, inplace=True)

x2 = x2.groupby(['Make'])['cnt'].agg('max').reset_index()
print(x2)

time2 = timeit.default_timer()
print("\ntime = ", (time2 - time1) )


      Make    cnt
0     AAUD      1
1     AAVH      2
2     ABAR      3
3      ABC      3
4     ABRI      1
5      ACC      1
6     ACCR      1
7     ACCU      3
8     ACDI      1
9      ACG      2
10    ACOR      1
11    ACRA      4
12    ACRU      1
13     ACU     10
14    ACUA      1
15    ACUR  13041
16    ADIA      1
17    ADVE      1
18    AERO      1
19    AFLA      1
20    AIRB      1
21    AIRE      4
22    AIRS      2
23    AIRX      1
24    AJAX      8
25    ALAX      1
26    ALEG      7
27    ALER      2
28    ALFA    202
29    ALGR      2
...    ...    ...
1439  WNBO      1
1440  WNDE      1
1441  WNEG      1
1442  WNES      2
1443  WNGB      1
1444  WNNE      1
1445  WNSS      1
1446  WOND      1
1447  WORH      1
1448  WORK      5
1449  WORL      1
1450  WRAN     10
1451  WRKH      2
1452  WRRC      1
1453    XL      1
1454  XLSP      1
1455   YAM      3
1456  YAMA    709
1457  YAMH      1
1458  YANB      1
1459  YONG      2
1460  ZEIM      2
1461  ZEMA      4
1462  ZHEN

### Results: SQLite 5.35s / Pandas 4.21s

# Problem (c) ----------------------------------------------------

In [20]:
# First, note the formatting of the Issue_date column
#df.head()


In [21]:
# Reformat it, before doing a comparison

# practice
#date = "2015-12-21T00:00:00"
#dt.datetime.strptime(date,"%Y-%m-%dT%H:%M:%S")

# Convert column 'Issue_Date' to a datetime object, so comparisons can be easily made
df['Issue_Date_new'] = pd.to_datetime( df['Issue_Date'], format="%Y-%m-%dT%H:%M:%S" )


In [22]:
#df.head()

### SQLite

In [23]:
cur.execute("DROP TABLE IF EXISTS temp1")

for row in cur.execute("CREATE TABLE temp1 AS SELECT Make, Issue_Date, strftime('%Y-%m-%dT%H:%M:%S',Issue_Date) AS Issue_Date_new FROM mytest WHERE Make IS NOT NULL"):
    print(row)
    

In [24]:
time1 = timeit.default_timer()

for row in cur.execute("SELECT Make, MIN(Issue_Date_new) FROM temp1 GROUP BY Make"):
    print(row)

time2 = timeit.default_timer()
print("\ntime = ", (time2 - time1) )
    


('AAUD', '2018-07-13T00:00:00')
('AAVH', '2018-03-29T00:00:00')
('ABAR', '2016-09-15T00:00:00')
('ABC', '2016-05-10T00:00:00')
('ABRI', '2017-04-18T00:00:00')
('ACC', '2015-12-23T00:00:00')
('ACCR', '2018-02-12T00:00:00')
('ACCU', '2015-04-07T00:00:00')
('ACDI', '2017-12-01T00:00:00')
('ACG', '2016-03-15T00:00:00')
('ACOR', '2018-03-07T00:00:00')
('ACRA', '2016-12-04T00:00:00')
('ACRU', '2018-08-22T00:00:00')
('ACU', '2016-12-19T00:00:00')
('ACUA', '2016-11-29T00:00:00')
('ACUR', '2010-06-09T00:00:00')
('ADIA', '2016-06-02T00:00:00')
('ADVE', '2016-11-01T00:00:00')
('AERO', '2017-07-14T00:00:00')
('AFLA', '2018-03-01T00:00:00')
('AIRB', '2018-10-16T00:00:00')
('AIRE', '2016-01-27T00:00:00')
('AIRS', '2015-02-12T00:00:00')
('AIRX', '2015-02-11T00:00:00')
('AJAX', '2015-05-21T00:00:00')
('ALAX', '2015-02-23T00:00:00')
('ALEG', '2017-09-10T00:00:00')
('ALER', '2015-03-15T00:00:00')
('ALFA', '2014-11-02T00:00:00')
('ALGR', '2016-12-23T00:00:00')
('ALJD', '2017-10-27T00:00:00')
('ALJO', '20

('DMW', '2018-10-28T00:00:00')
('DODG', '2011-09-10T00:00:00')
('DOLI', '2018-11-24T00:00:00')
('DOLP', '2015-08-19T00:00:00')
('DOOH', '2016-12-29T00:00:00')
('DORS', '2017-04-06T00:00:00')
('DOT', '2015-09-02T00:00:00')
('DOVE', '2018-09-20T00:00:00')
('DPLM', '2015-09-30T00:00:00')
('DRAG', '2016-01-31T00:00:00')
('DRLN', '2016-01-29T00:00:00')
('DRME', '2018-12-14T00:00:00')
('DRMR', '2015-10-10T00:00:00')
('DROS', '2015-11-04T00:00:00')
('DTAM', '2017-08-23T00:00:00')
('DUCA', '2015-01-13T00:00:00')
('DUES', '2015-01-14T00:00:00')
('DUHN', '2015-08-04T00:00:00')
('DUNH', '2015-08-06T00:00:00')
('DUTC', '2016-12-06T00:00:00')
('DUTS', '2017-05-10T00:00:00')
('DYNA', '2015-06-02T00:00:00')
('EAGL', '2015-12-03T00:00:00')
('EAST', '2016-10-13T00:00:00')
('ECLP', '2016-05-11T00:00:00')
('EFAB', '2015-11-13T00:00:00')
('EGIL', '2014-12-23T00:00:00')
('EGLE', '2015-01-06T00:00:00')
('EJET', '2017-05-20T00:00:00')
('EL', '2016-12-18T00:00:00')
('EL D', '2017-11-29T00:00:00')
('ELAN', '20

('MERY', '2015-10-01T00:00:00')
('MERZ', '2010-10-20T00:00:00')
('MESC', '2015-10-10T00:00:00')
('METR', '2015-04-30T00:00:00')
('MEZ', '2012-12-17T00:00:00')
('MEZD', '2016-02-24T00:00:00')
('MG', '2015-10-21T00:00:00')
('MGM', '2016-01-10T00:00:00')
('MGR', '2017-02-18T00:00:00')
('MH', '2015-12-30T00:00:00')
('MIAI', '2016-01-25T00:00:00')
('MIAT', '2016-11-19T00:00:00')
('MICE', '2018-12-13T00:00:00')
('MID', '2015-08-17T00:00:00')
('MIDA', '2015-03-31T00:00:00')
('MIDS', '2017-06-01T00:00:00')
('MIHI', '2018-05-03T00:00:00')
('MILE', '2017-04-13T00:00:00')
('MILL', '2016-05-28T00:00:00')
('MIN', '2015-05-14T00:00:00')
('MINC', '2015-06-02T00:00:00')
('MINI', '2014-10-29T00:00:00')
('MINN', '2015-01-08T00:00:00')
('MINT', '2015-06-19T00:00:00')
('MIS', '2018-12-15T00:00:00')
('MISH', '2018-05-29T00:00:00')
('MISS', '2015-08-28T00:00:00')
('MIST', '2017-01-02T00:00:00')
('MISV', '2018-01-07T00:00:00')
('MIT', '2016-11-22T00:00:00')
('MITC', '2015-08-21T00:00:00')
('MITE', '2018-08-1

('TOYOT', '2018-02-24T00:00:00')
('TOYT', '2014-10-30T00:00:00')
('TRAC', '2015-09-01T00:00:00')
('TRAI', '2015-06-24T00:00:00')
('TRAN', '2015-06-01T00:00:00')
('TRAV', '2015-06-17T00:00:00')
('TRBZ', '2017-08-17T00:00:00')
('TRI', '2016-02-12T00:00:00')
('TRIN', '2017-05-30T00:00:00')
('TRIS', '2015-11-23T00:00:00')
('TRIU', '2014-10-31T00:00:00')
('TRJY', '2015-01-18T00:00:00')
('TRK', '2017-08-09T00:00:00')
('TRLC', '2018-04-17T00:00:00')
('TRLM', '2015-08-01T00:00:00')
('TRLR', '2016-01-15T00:00:00')
('TROO', '2018-12-22T00:00:00')
('TROU', '2017-07-17T00:00:00')
('TRQN', '2015-02-04T00:00:00')
('TRQU', '2018-04-12T00:00:00')
('TRTT', '2017-09-13T00:00:00')
('TRUC', '2017-04-01T00:00:00')
('TRUM', '2015-01-18T00:00:00')
('TRUP', '2017-08-10T00:00:00')
('TRVC', '2017-01-06T00:00:00')
('TRVL', '2017-09-06T00:00:00')
('TRWS', '2016-09-27T00:00:00')
('TSHR', '2015-07-04T00:00:00')
('TSLA', '2015-01-14T00:00:00')
('TSLR', '2015-03-19T00:00:00')
('TSMR', '2014-10-31T00:00:00')
('TTC', '

### Pandas

In [25]:
time1 = timeit.default_timer()

filt = ~df['Make'].isnull()
df2 = df.loc[ filt, ['Make','Issue_Date','Issue_Date_new'] ]

y = df2.groupby(['Make'])['Issue_Date_new'].agg('min').reset_index()
print(y)

time2 = timeit.default_timer()
print("\ntime = ", (time2 - time1) )


      Make Issue_Date_new
0     AAUD     2018-07-13
1     AAVH     2018-03-29
2     ABAR     2016-09-15
3      ABC     2016-05-10
4     ABRI     2017-04-18
5      ACC     2015-12-23
6     ACCR     2018-02-12
7     ACCU     2015-04-07
8     ACDI     2017-12-01
9      ACG     2016-03-15
10    ACOR     2018-03-07
11    ACRA     2016-12-04
12    ACRU     2018-08-22
13     ACU     2016-12-19
14    ACUA     2016-11-29
15    ACUR     2010-06-09
16    ADIA     2016-06-02
17    ADVE     2016-11-01
18    AERO     2017-07-14
19    AFLA     2018-03-01
20    AIRB     2018-10-16
21    AIRE     2016-01-27
22    AIRS     2015-02-12
23    AIRX     2015-02-11
24    AJAX     2015-05-21
25    ALAX     2015-02-23
26    ALEG     2017-09-10
27    ALER     2015-03-15
28    ALFA     2014-11-02
29    ALGR     2016-12-23
...    ...            ...
1447  WNBO     2017-04-30
1448  WNDE     2017-01-11
1449  WNEG     2017-08-25
1450  WNES     2015-06-03
1451  WNGB     2018-08-10
1452  WNNE     2015-05-08
1453  WNSS  

### Results:  SQLite 2.85s / Pandas 1.39s

# EXTRA Problem -------------------------------------------------------

### Appears to be California data.  So "out of state" means non-CA license plates.  I say this because counts are high for CA plates, and street addresses include famous CA streets.  Latitude/Longitude are in non-standard format...

In [26]:
# (It's easy to compute the mapping bertween standard Longitude/Latitude and what they're using (GPS?).)

In [27]:
# Only look at Make != null
filt = ~df['Make'].isnull() & ~df['RP_State_Plate'].isnull() & ~df['Plate_Expiry_Date'].isnull()
df3 = df.loc[ filt, ['RP_State_Plate','Issue_Date','Issue_Date_new','Plate_Expiry_Date'] ]
df3.head(10)


,RP_State_Plate,Issue_Date,Issue_Date_new,Plate_Expiry_Date
4,CA,2015-09-15T00:00:00,2015-09-15,200316.0
6,CA,2015-12-17T00:00:00,2015-12-17,201605.0
10,CA,2015-12-22T00:00:00,2015-12-22,201511.0
11,CA,2015-12-22T00:00:00,2015-12-22,201701.0
14,CA,2015-12-27T00:00:00,2015-12-27,201605.0
19,CA,2015-12-22T00:00:00,2015-12-22,201606.0
21,CA,2015-12-18T00:00:00,2015-12-18,201509.0
22,CA,2015-12-18T00:00:00,2015-12-18,201511.0
25,CA,2015-12-22T00:00:00,2015-12-22,201610.0
29,CA,2015-12-22T00:00:00,2015-12-22,201601.0


In [28]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3960997 entries, 4 to 8726013
Data columns (total 4 columns):
RP_State_Plate       object
Issue_Date           object
Issue_Date_new       datetime64[ns]
Plate_Expiry_Date    float64
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 151.1+ MB


In [29]:
# get rid of decimal part
df3['Plate_Expiry_Date'] = df3['Plate_Expiry_Date'].astype(int).astype(str)


In [30]:
# practice
#"201902"[0:4]  #year
#"201902"[4:6]  #month


In [31]:
# This is one way to count the number of times the date is incorrectly formatted
df3['Expiry_month'] = df3['Plate_Expiry_Date'].apply(lambda x: x[4:6] if len(x)==6 else "0" )
df3['Expiry_year'] = df3['Plate_Expiry_Date'].apply(lambda x: x[0:4] if len(x)==6 else "0" )


In [32]:
df3['Expiry_month'] = df3['Expiry_month'].astype(int)
df3['Expiry_year'] = df3['Expiry_year'].astype(int)


In [33]:
df3.head(10)

,RP_State_Plate,Issue_Date,Issue_Date_new,Plate_Expiry_Date,Expiry_month,Expiry_year
4,CA,2015-09-15T00:00:00,2015-09-15,200316,16,2003
6,CA,2015-12-17T00:00:00,2015-12-17,201605,5,2016
10,CA,2015-12-22T00:00:00,2015-12-22,201511,11,2015
11,CA,2015-12-22T00:00:00,2015-12-22,201701,1,2017
14,CA,2015-12-27T00:00:00,2015-12-27,201605,5,2016
19,CA,2015-12-22T00:00:00,2015-12-22,201606,6,2016
21,CA,2015-12-18T00:00:00,2015-12-18,201509,9,2015
22,CA,2015-12-18T00:00:00,2015-12-18,201511,11,2015
25,CA,2015-12-22T00:00:00,2015-12-22,201610,10,2016
29,CA,2015-12-22T00:00:00,2015-12-22,201601,1,2016


In [34]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3960997 entries, 4 to 8726013
Data columns (total 6 columns):
RP_State_Plate       object
Issue_Date           object
Issue_Date_new       datetime64[ns]
Plate_Expiry_Date    object
Expiry_month         int64
Expiry_year          int64
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 211.5+ MB


In [35]:
filt2 = (df3['Expiry_month'] > 0) & (df3['Expiry_month'] < 13) & (df3['Expiry_year'] > 2000) & (df3['Expiry_year'] < 2040)
df4 = df3.loc[ filt2 ].copy()
df4.head(10)


,RP_State_Plate,Issue_Date,Issue_Date_new,Plate_Expiry_Date,Expiry_month,Expiry_year
6,CA,2015-12-17T00:00:00,2015-12-17,201605,5,2016
10,CA,2015-12-22T00:00:00,2015-12-22,201511,11,2015
11,CA,2015-12-22T00:00:00,2015-12-22,201701,1,2017
14,CA,2015-12-27T00:00:00,2015-12-27,201605,5,2016
19,CA,2015-12-22T00:00:00,2015-12-22,201606,6,2016
21,CA,2015-12-18T00:00:00,2015-12-18,201509,9,2015
22,CA,2015-12-18T00:00:00,2015-12-18,201511,11,2015
25,CA,2015-12-22T00:00:00,2015-12-22,201610,10,2016
29,CA,2015-12-22T00:00:00,2015-12-22,201601,1,2016
30,CA,2015-12-22T00:00:00,2015-12-22,201601,1,2016


In [36]:
#df4._is_view, df4._is_copy
df4.shape

(3663555, 6)

In [37]:
df4['Expiry_new'] = pd.to_datetime( df4['Plate_Expiry_Date'], format="%Y%m" )
df4.head(10)


,RP_State_Plate,Issue_Date,Issue_Date_new,Plate_Expiry_Date,Expiry_month,Expiry_year,Expiry_new
6,CA,2015-12-17T00:00:00,2015-12-17,201605,5,2016,2016-05-01
10,CA,2015-12-22T00:00:00,2015-12-22,201511,11,2015,2015-11-01
11,CA,2015-12-22T00:00:00,2015-12-22,201701,1,2017,2017-01-01
14,CA,2015-12-27T00:00:00,2015-12-27,201605,5,2016,2016-05-01
19,CA,2015-12-22T00:00:00,2015-12-22,201606,6,2016,2016-06-01
21,CA,2015-12-18T00:00:00,2015-12-18,201509,9,2015,2015-09-01
22,CA,2015-12-18T00:00:00,2015-12-18,201511,11,2015,2015-11-01
25,CA,2015-12-22T00:00:00,2015-12-22,201610,10,2016,2016-10-01
29,CA,2015-12-22T00:00:00,2015-12-22,201601,1,2016,2016-01-01
30,CA,2015-12-22T00:00:00,2015-12-22,201601,1,2016,2016-01-01


In [38]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3663555 entries, 6 to 8726013
Data columns (total 7 columns):
RP_State_Plate       object
Issue_Date           object
Issue_Date_new       datetime64[ns]
Plate_Expiry_Date    object
Expiry_month         int64
Expiry_year          int64
Expiry_new           datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(3)
memory usage: 223.6+ MB


### Now compute probabilities...

In [39]:
# number in CA
num_CA = (df4['RP_State_Plate'] == 'CA').sum()
num_CA


3443360

In [40]:
# number NOT in CA
num_notCA = (df4['RP_State_Plate'] != 'CA').sum()
num_notCA


220195

In [41]:
# number w/ expired plates at time of ticket, registered in CA
filt3 = (df4['Issue_Date_new'] > df4['Expiry_new']) & (df4['RP_State_Plate'] == 'CA')
df5 = df4.loc[ filt3 ]
num_CA_expired = df5.shape[0]
num_CA_expired


756920

In [42]:
# number w/ expired plates at time of ticket, NOT registered in CA
filt3 = (df4['Issue_Date_new'] > df4['Expiry_new']) & (df4['RP_State_Plate'] != 'CA')
df5 = df4.loc[ filt3 ]
num_notCA_expired = df5.shape[0]
num_notCA_expired


51664

In [43]:
# Probability for a CA driver to have expired plates at time of ticket
num_CA_expired / (1.0*num_CA)


0.2198201756423958

In [44]:
# Probability for a non-CA driver to have expired plates at time of ticket
num_notCA_expired / (1.0*num_notCA)


0.2346283975567111

### Conclusion: wrt ticketing in CA, more likely for a non-CA driver to have expired plates

In [45]:
# This difference is likely "significant", since the counts are high.  Easy to check, if desired.


In [ ]:
# TO DO: check that license plates are valid (compare to a list with valid abbreviations)
